In [1]:
import math
import statistics
import pandas as pd
import numpy as np
import altair as alt
import vl_convert as vlc

In [2]:
# PARSE METADATA

# NOTE: change me
md_fp = ''
md = pd.read_csv(md_fp, sep='\t', dtype={'sample-id': str})

# remove comment lines
md = md[~ md['sample-id'].str.startswith('#')]

md = md.rename({'sample-id': 'sample_id'}, axis=1)

md = md[['sample_id', 'Bucket', 'Composting Time Point', 'SampleType']]

# drop nan & week-0 rows of HEC
md = md[~(
    (md['Composting Time Point'] == 'Human Excrement Compost') &
    (md['Composting Time Point'].isna() | md['Composting Time Point'] > 0)
)]

# keep only HE and HEC observations
md = md[md['SampleType'].isin(['Human Excrement Compost', 'Human Excrement'])]

In [3]:
# PARSE BACTQUANT DATA

# NOTE: change me
bq_fp = ''
bq = pd.read_excel(
    bq_fp,
    sheet_name='MasterFile',
    dtype={'Sample ID': str}
)
bq = bq[['Sample ID', 'Copy #']]
bq = bq.rename(
    {'Sample ID': 'sample_id', 'Copy #': 'bq_copy_number'},
    axis=1
)

In [4]:
# PARSE QPCR DATA, which exist in multiple sheets

# NOTE: change me
ecoli_cperf_fp = ''

sheets = pd.read_excel(ecoli_cperf_fp, sheet_name=None)
ecoli_cperf = pd.DataFrame()
for sheet_name, sheet in sheets.items():
    ecoli_cperf = pd.concat([ecoli_cperf, sheet], axis=0)

ecoli_cperf['Sample Name'] = ecoli_cperf['Sample Name'].astype(str)

ecoli_cperf = ecoli_cperf[['Sample Name', 'Target Name', 'Quantity Mean', 'Quantity SD']]
ecoli_cperf = ecoli_cperf.rename(
    {
        'Sample Name': 'sample_id', 'Target Name': 'target_name',
        'Quantity Mean': 'quantity_mean', 'Quantity SD': 'quantity_sd'
    },
    axis=1
)

In [5]:
# averaging already done in excel; keep only one row per triplicate set
ecoli_cperf = ecoli_cperf.groupby(['sample_id', 'target_name']).head(1)

# replace missing quantities with 0 for convenience
# IMPORTANT: we do not know whether these quantifications were 0,
# only that there are less than the limit of detection
ecoli_cperf = ecoli_cperf.fillna({'quantity_mean': 0})

# drop NTCs
ecoli_cperf = ecoli_cperf[ecoli_cperf.sample_id != 'NTC']

In [6]:
# TODO: looks like there were some data entry errors for the assay name
print( ecoli_cperf.target_name.value_counts() )

# get rid of them for now
ecoli_cperf = ecoli_cperf[ ecoli_cperf.target_name.isin(['EcoliIUD', 'Cperf23S']) ]

target_name
Cperf23S           1052
EcoliIUD           1052
EcoliIUD3.041E1       1
EcoliIUD3.041E2       1
EcoliIUD3.041E3       1
EcoliIUD3.041E4       1
EcoliIUD3.041E5       1
EcoliIUD3.041E6       1
EcoliIUD3.041E7       1
Cperf23S3.041         1
Cperf23S3.041E1       1
Cperf23S3.041E2       1
Cperf23S3.041E3       1
Cperf23S3.041E4       1
Cperf23S3.041E5       1
Cperf23S3.041E6       1
Cperf23S3.041E7       1
EcoliIUD3.041         1
Name: count, dtype: int64


In [7]:
# PIVOT ASSAY MEASUREMENTS TO COLUMNS

ecoli_cperf = ecoli_cperf.pivot(index='sample_id', columns='target_name')
ecoli_cperf.columns = ecoli_cperf.columns.to_flat_index().map(lambda c: '_'.join(c))
ecoli_cperf.reset_index(inplace=True)

In [8]:
# MERGE ECOLI/CPERF WITH METADATA

ecoli_cperf_md = ecoli_cperf.merge(md, how='right', on='sample_id')

In [9]:
# ADD LIMIT OF QUANTIFICATION AND LIMIT OF DETECTION VALUES

ecoli_cperf_md['loq'] = 302
ecoli_cperf_md['lod'] = 30.2

In [10]:
# NORMALIZE ECOLI, CPERF QUANTIFICATIONS BY BACTQUANT

merged = ecoli_cperf_md.merge(right=bq, on='sample_id', how='inner')

merged['cperf_normalized'] = merged.quantity_mean_Cperf23S / merged.bq_copy_number
merged['ecoli_normalized'] = merged.quantity_mean_EcoliIUD / merged.bq_copy_number

# normalize limit of quantification as well
merged['loq_normalized'] = merged.loq / merged.bq_copy_number

ecoli_cperf_md = merged

In [11]:
# RECAST WEEK AND BUCKET COLUMNS TO INTEGER

ecoli_cperf_md['Bucket'] = ecoli_cperf_md['Bucket'].astype('Int64')
ecoli_cperf_md['Composting Time Point'] = ecoli_cperf_md['Composting Time Point'].astype('Int64')

In [12]:
# SUBSET TO BUCKETS OF INTEREST

buckets_to_keep = list(range(1, 8)) + list(range(9, 17))
ecoli_cperf_md = ecoli_cperf_md[ecoli_cperf_md['Bucket'].isin(buckets_to_keep)]

ecoli_cperf_md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sample_id               815 non-null    object 
 1   quantity_mean_Cperf23S  812 non-null    float64
 2   quantity_mean_EcoliIUD  812 non-null    float64
 3   quantity_sd_Cperf23S    725 non-null    float64
 4   quantity_sd_EcoliIUD    409 non-null    float64
 5   Bucket                  815 non-null    Int64  
 6   Composting Time Point   770 non-null    Int64  
 7   SampleType              815 non-null    object 
 8   loq                     815 non-null    int64  
 9   lod                     815 non-null    float64
 10  bq_copy_number          813 non-null    float64
 11  cperf_normalized        810 non-null    float64
 12  ecoli_normalized        810 non-null    float64
 13  loq_normalized          813 non-null    float64
dtypes: Int64(2), float64(9), int64(1), object(

In [13]:
# RENAME QUANTITY COLUMNS

ecoli_cperf_md = ecoli_cperf_md.rename(
    {
        'quantity_mean_EcoliIUD': 'ecoli_quantity_mean',
        'quantity_mean_Cperf23S': 'cperf_quantity_mean'
    },
    axis=1,
)

In [14]:
# CALCULATE MOVING AVERAGES

# sort by week for moving average calculation
ecoli_cperf_md = ecoli_cperf_md.sort_values('Composting Time Point')

# averaging period
window = 4

# standard moving averages
ecoli_cperf_md['ecoli_mov_avg'] = (
    ecoli_cperf_md.groupby(['Bucket', 'SampleType'])['ecoli_quantity_mean']
    .transform(lambda x: x.rolling(window=window, min_periods=1).mean())
)
ecoli_cperf_md['cperf_mov_avg'] = (
    ecoli_cperf_md.groupby(['Bucket', 'SampleType'])['cperf_quantity_mean']
    .transform(lambda x: x.rolling(window=window, min_periods=1).mean())
)

# bactquant-normalized moving averages
ecoli_cperf_md['ecoli_normalized_mov_avg'] = (
  ecoli_cperf_md.groupby(['Bucket', 'SampleType'])['ecoli_normalized']
    .transform(lambda x: x.rolling(window=window, min_periods=1).mean())
)
ecoli_cperf_md['cperf_normalized_mov_avg'] = (
  ecoli_cperf_md.groupby(['Bucket', 'SampleType'])['cperf_normalized']
    .transform(lambda x: x.rolling(window=window, min_periods=1).mean())
)

In [15]:
# ADD PSEUDO COUNTS FOR LOG SCALE

def calculate_pseudo_count(df, column):
    minimum = df[df[column] > 0][column].min()
    pseudo_count_exp = math.floor(math.log10(minimum)) - 1
    return 1 * 10**(pseudo_count_exp)

replace = [
    'ecoli_mov_avg', 'ecoli_quantity_mean', 'cperf_mov_avg', 'cperf_quantity_mean',
    'ecoli_normalized_mov_avg', 'ecoli_normalized', 'cperf_normalized_mov_avg', 'cperf_normalized',
]
for replace_column in replace:
    pseudo_count = calculate_pseudo_count(ecoli_cperf_md, replace_column)
    ecoli_cperf_md.loc[:, replace_column] += pseudo_count

In [16]:
# FILL FECAL SAMPLE TIME POINTS AS 0

ecoli_cperf_md.loc[ecoli_cperf_md['SampleType'] == 'Human Excrement', 'Composting Time Point'] = \
    ecoli_cperf_md.loc[ecoli_cperf_md['SampleType'] == 'Human Excrement', 'Composting Time Point'].fillna(0)

In [17]:
# RENAME qPCR COLUMNS FOR BETTER LEGIBILITY
renames = {
    'cperf_mov_avg': 'C perfringens Moving Average',
    'ecoli_mov_avg': 'E coli Moving Average',
    'cperf_normalized_mov_avg': 'C perfringens Moving Average (Normalized)',
    'ecoli_normalized_mov_avg': 'E coli Moving Average (Normalized)',
    'loq': 'Limit of Quantification',
    'loq_normalized': 'Limit of Quantification (Normalized)',
    'SampleType': 'Sample Type',
}
ecoli_cperf_md = ecoli_cperf_md.rename(renames, axis=1)

In [18]:
# NOTE: change me
y_title = 'E coli Copy Number (Normalized)'
line_y_value = 'E coli Moving Average (Normalized)'
point_y_value = 'ecoli_normalized'

pseudo_count = ecoli_cperf_md[line_y_value].min()

line = alt.Chart(ecoli_cperf_md).transform_fold(
    [line_y_value, 'Limit of Quantification (Normalized)'],
    as_=['Line', 'Amount'],
).mark_line().encode(
    x=alt.X('Composting Time Point').scale(domain=(0, 52), nice=False),
    y=alt.Y('Amount', type='quantitative').scale(type='log').title(y_title),
    color='Line:N',
    strokeDash='Line:N',
).transform_filter(
    (alt.datum['Sample Type'] == 'Human Excrement Compost')
)

point = alt.Chart(ecoli_cperf_md).mark_point(
    filled=True,
    size=75,
    color='black',
    fillOpacity=0.85
).encode(
    x=alt.X('Composting Time Point'),
    y=alt.Y(point_y_value).scale(type='log'),
    shape='Sample Type'
).transform_filter(
    (alt.datum['Sample Type'] == 'Human Excrement')
    #(alt.datum.ecoli_quantity_mean < 1_000_000)
)

alt.layer(line, point).facet(
    facet='Bucket',
    columns=5,
    title=alt.Title(
        f'{y_title}, By Bucket',
        subtitle=[
            'moving average, period of 4',
            f'pseudo-count of {pseudo_count} added to all data points',
        ]
    )
).resolve_scale(
    y='independent',
    x='independent'
).configure_legend(
    labelLimit=0,
    titleFontSize=18,
    labelFontSize=16,
    orient='bottom',
    padding=25
).configure_axis(
    labelFontSize=15,
    titleFontSize=15,
).configure_axisY(
    format='e'
).configure_header(
    titleFontSize=16,
    labelFontSize=15
).configure_title(
    fontSize=20,
    subtitleFontSize=18
)

alt.FacetChart(...)

In [19]:
# BACTQUANT ONLY PLOTS

bq = alt.Chart(ecoli_cperf_md).mark_line().encode(
    x=alt.X('Composting Time Point').scale(domain=(0, 52), nice=False),
    y=alt.Y('bq_copy_number').title('BactQuant Copy Number').scale(type='log'),
).transform_filter(
    (alt.datum['Sample Type'] == 'Human Excrement Compost')
).facet(
    facet='Bucket',
    columns=5,
    title=alt.Title(
        'BactQuant Copy Number, By Bucket',
        #subtitle=['moving average, period=4',]
    )
).resolve_scale(
    y='independent',
    x='independent'
).configure_axis(
    labelFontSize=15,
    titleFontSize=15,
).configure_axisY(
    format='e'
).configure_header(
    titleFontSize=16,
    labelFontSize=15
).configure_title(
    fontSize=20
)

bq

alt.FacetChart(...)